# Load Dataset

## NMT Dataset

In [3]:
##ParaNMT
with open('./Datasets/para-nmt-5m-processed.txt', encoding="utf8") as file:
    nmt = file.readlines()

In [4]:
nmt_sentences = list()
for idx, sentence in enumerate(nmt):
    nmt_sentences.append(nmt[idx].split("\t")[0]) #non-parallel data
nmt_sentences[:5] #list

["so , unless that 's gon na be feasible , then ...",
 'of course you did .',
 "by now , singh 's probably been arrested .",
 'not our shit . i swear .',
 '“ why not ?']

In [70]:
nmt_sentences

["so , unless that 's gon na be feasible , then ...",
 'of course you did .',
 "by now , singh 's probably been arrested .",
 'not our shit . i swear .',
 '“ why not ?',
 'instead , he folded his arms and cocked his head , as if encouraging—or daring—kaye to continue .',
 "an old man 's mistake … '",
 'he reached for a microphone and i could hear his voice booming faintly throughout the ship .',
 "she said it with such simple dignity mat a lump caught in gar-ion 's throat .",
 'he loved that little man , by the way .',
 'provide the following information',
 "i 've been very , very lucky .",
 'you would turn from the gods , and their guidance ?',
 'but i did not give her the love that she ... deserved .',
 "well , then , i 'm kind of gon na rub this dinner in her face next time i see her .",
 'why are you carrying around grass ?',
 "it 's like , gross , yet oddly delicious .",
 "not as much as i 'd like",
 "there 's no doubt in my mind that this is the right group of men but we still ha

In [5]:
len(nmt_sentences) #5370128 sentences

5370128

## Configuration

In [6]:
import nltk
# benepar.download('benepar_en3')
import benepar, spacy
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

def is_paren(tok):
    return tok == ")" or tok == "("

def getleaf(tree):
    nonleaves = ''
    for w in str(tree).replace('\n', '').split():
        w = w.replace('(', '( ').replace(')', ' )')
        nonleaves += w + ' '
    
    leaves = []
    arr = nonleaves.split()
    for n, i in enumerate(arr):
        if n + 1 < len(arr):
            tok1 = arr[n]
            tok2 = arr[n + 1]
            if not is_paren(tok1) and not is_paren(tok2):
                leaves.append(arr[n])

    return leaves

def deleaf(tree):
    nonleaves = ''
    for w in str(tree).replace('\n', '').split():
        w = w.replace('(', '( ').replace(')', ' )')
        nonleaves += w + ' '

    arr = nonleaves.split()
    for n, i in enumerate(arr):
        if n + 1 < len(arr):
            tok1 = arr[n]
            tok2 = arr[n + 1]
            if not is_paren(tok1) and not is_paren(tok2):
                arr[n + 1] = ""

    nonleaves = " ".join(arr)
    return nonleaves.split()

import pickle
with open("./data/dictionary.pkl", "rb") as file:
    dictionary = pickle.load(file)

#Setence to syntax
def constituency_parser(text):
    doc = nlp(text)
    sent = list(doc.sents)[0]
    return "(ROOT "+sent._.parse_string+")"

#syntax to syntatic tokenzier
from nltk import ParentedTree
def parser_tokenizer(synt_):
    synt_ = ParentedTree.fromstring(synt_)
    synt_ = deleaf(synt_)
    # synt_ = [f'<{w}>' for w in synt_]
    synt_ = [dictionary.word2idx[f"<{w}>"] for w in synt_ if f"<{w}>" in dictionary.word2idx]
    return synt_

In [7]:
#Tokenizer BPE
from subwordnmt.apply_bpe import BPE, read_vocabulary
import codecs

# load bpe codes
bpe_codes = codecs.open('./data/bpe.codes', encoding='utf-8')
bpe_vocab = codecs.open('./data/vocab.txt', encoding='utf-8')
bpe_vocab = read_vocabulary(bpe_vocab, 50)
bpe = BPE(bpe_codes, '@@', bpe_vocab, None)

def bpe_tokenizer(sent_, target = False):
 # bpe segment and convert to tensor
    sent_ = bpe.segment(sent_).split()
    sent_ = [dictionary.word2idx[w] if w in dictionary.word2idx else dictionary.word2idx["<unk>"] for w in sent_]
    if target:
        sent_ = [dictionary.word2idx["<sos>"]] + sent_ + [dictionary.word2idx["<eos>"]]
    return sent_

# def pos_tag(sent_):
#     return [token.ent_type_ if token.ent_type_ else "" for token in nlp(sent_)]

# def dependency_parser(sent_):
#     return [token.dep_ for token in nlp(sent_)]

In [71]:
import numpy as np
import tqdm

import pickle
with open('synt_vocab.pkl', 'rb') as f:
    synt_vocab = pickle.load(f)

print(synt_vocab)

def bow(synt_):
    synt_ = ParentedTree.fromstring(synt_)
    synt_ = deleaf(synt_)
    synt_bow = np.ones(74)
    #non-inculde <s> , <pad>, </s>
    #start from '(': 3 instead
    for tag in synt_:
        if tag != '<sos>' and tag != '<eos>':
            try:
                synt_bow[synt_vocab[tag]-3] += 1
            except:
                pass
    return synt_bow

{'<s>': 0, '<pad>': 1, '</s>': 2, '(': 3, 'ROOT': 4, 'S': 5, 'ADVP': 6, 'RB': 7, ')': 8, ',': 9, 'INTJ': 10, 'UH': 11, 'FW': 12, 'VP': 13, 'VBP': 14, 'ADJP': 15, 'JJ': 16, ':': 17, 'NP': 18, 'NN': 19, '.': 20, 'PRP': 21, 'VBD': 22, 'VBG': 23, 'TO': 24, 'VB': 25, 'PRT': 26, 'FRAG': 27, 'SBAR': 28, 'IN': 29, 'QP': 30, 'VBZ': 31, 'CD': 32, 'PP': 33, 'VBN': 34, 'DT': 35, 'CC': 36, 'NNS': 37, 'PRP$': 38, 'WHNP': 39, 'WP': 40, 'LS': 41, 'NNP': 42, 'SINV': 43, 'PRN': 44, '``': 45, "''": 46, 'JJR': 47, 'WDT': 48, 'POS': 49, 'MD': 50, 'SQ': 51, 'SBARQ': 52, 'WHADVP': 53, 'WRB': 54, 'RP': 55, 'EX': 56, 'JJS': 57, 'X': 58, 'LST': 59, '-LRB-': 60, '-RRB-': 61, 'RBS': 62, 'UCP': 63, 'RBR': 64, 'WHPP': 65, 'PDT': 66, 'WHADJP': 67, 'NX': 68, 'CONJP': 69, '$': 70, 'WP$': 71, '#': 72, 'SYM': 73, 'NNPS': 74, 'RRC': 75, 'NAC': 76}


# Adversary Generator

In [72]:
from tqdm import tqdm

def prepare_paraphrase_dataset(sentences):
    lists_ = list()
    for idx in tqdm(range(len(sentences))):

        sent_ = bpe_tokenizer(sentences[idx])
        synt_ = parser_tokenizer(constituency_parser(sentences[idx]))
        trg_  = bpe_tokenizer(sentences[idx], target = True)
        bow_  = bow(constituency_parser(sentences[idx]))
        lists_.append((sent_, synt_, trg_, bow_))
    return lists_

In [73]:
nmt_sentences[8]

"she said it with such simple dignity mat a lump caught in gar-ion 's throat ."

In [75]:
sentences_1000 = prepare_paraphrase_dataset(nmt_sentences[:1000])

100%|██████████| 1000/1000 [02:57<00:00,  5.63it/s]


In [76]:
import pickle

with open('./data/adversary_1000.pkl', 'wb') as f:
    pickle.dump(sentences_1000, f, protocol=pickle.HIGHEST_PROTOCOL)